In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# googleplaystore.csv: https://drive.google.com/file/d/1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ/view?usp=drive_link
# googleplaystore_user_reviews.csv: https://drive.google.com/file/d/1XX4S3LUe_bVB94g5H2tv5rhEANeSX_sO/view?usp=drive_link
downloaded = drive.CreateFile({'id':"1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ"})
downloaded.GetContentFile('googleplaystore.csv')
downloaded1 = drive.CreateFile({'id':"1XX4S3LUe_bVB94g5H2tv5rhEANeSX_sO"})
downloaded1.GetContentFile('reviews.csv')


In [4]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context -> variable que vamos a utilizar
sc = spark.sparkContext

A Tomás le gustan los juegos de acción y nos pidió que le ofrezcamos el top 3 de juegos de acción, generando el ranking según la cantidad de reviews positivas que hayan recibido. Devolver algunas referencias (al azar) de cada uno de ellos, para ayudarlo a decidir qué juego bajarse.(⭐⭐)


In [ ]:
sqlContext = SQLContext(sc)
df_googleplaystore = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
df_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('reviews.csv', header=True, inferSchema=True)
rdd_googleplaystore = df_googleplaystore.rdd
rdd_reviews = df_reviews.rdd

separo y guardo en una lista los generos (estan separados por ';') para luego verficar cuales son del genero 'Accion'. Luego convierto la lista a como estaba antes (un string separando los generos con ';').

In [19]:
clean_rdd = rdd_googleplaystore.map(lambda x: (x['App'], x['Genres'].split(';')))
action_rdd = clean_rdd.filter(lambda x: 'Action' in x[1] ).map(lambda x: (x[0], ';'.join(x[1])))

Guardo en cada registro una tupla del tipo: (app, (tiene sentiment positivo, review)) en caso de tener sentiment positivo se guarda un 1 y sino un 0.

In [20]:
formatted_reviews = rdd_reviews.map(lambda x: (x['App'], (1 if x['Sentiment'] == 'Positive' else 0, x['Translated_Review'])))

Como la consigna dice referencia al azar --> se asume que puede ser una review negativa la que se le puede llegar a mostrar a Tomas. Se elige una que no sea nan

In [21]:
positive_reviews_for_app = formatted_reviews.reduceByKey(lambda x, y: (x[0] + y[0], x[1] if not pd.isnull(x[1]) else y[1]))

Elimino registros con misma clave (son la misma app)


In [22]:
merged_rdd = action_rdd.join(positive_reviews_for_app).distinct()

Obtengo los primeros 3 de acuerdo a la cantidad de reviews positivas

In [23]:
top_accion = merged_rdd.takeOrdered(3, lambda x: -x[1][1][0])
for juego in top_accion:
  print(f"{juego[0]}, Random review: {juego[1][1][1]}")

Helix Jump, Random review: Actually really enjoy playing game, however I paid upgraded version without ads still continued get ads. Emailed developer week ago along attached confirmation purchase never got response, therefore I'm giving star. Just lot downloads mean overlook people support game; especially ones voluntarily pay upgraded version.
Bowmasters, Random review: Really like game far. Great gameplay lots fun. Edit: I took away star constant asking rate even I've already rated taking away chances chests. Just seems excessive. It's good game, people need badgered rating.
Garena Free Fire, Random review: Please add bolt action Sniper rifles. That would great. The game way easy. And also please decrease amount vehicles per game? Or reduce vehicles except monster truck (because airdrop vehicle). The game way easy I think. Just increase recoil little players say made hard. Nice game. Keep Garena.
